In [12]:
%pip install numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np



Note: you may need to restart the kernel to use updated packages.


In [13]:
df = pd.read_csv('train.csv')

df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [15]:
train_df = df.copy()
test_df = pd.read_csv('test.csv')


In [16]:
from keras.utils import to_categorical

X = train_df.drop('label', axis=1).values
y = train_df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Redimensionar y normalizar
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Convertir etiquetas a one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [27]:
import torch
import torch.nn as nn

class ImprovedNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Convolutional layers
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        # Fully connected layers
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128 * 3 * 3, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        # Ensure input is in the right shape [batch_size, channels, height, width]
        if len(x.shape) == 3:
            x = x.unsqueeze(1)
        
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc_layers(x)
        return x

# Training function with validation
def train_model(model, train_loader, val_loader, epochs, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
        train_acc = 100. * correct / total
        
        # Validation phase
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
        
        val_acc = 100. * correct / total
        scheduler.step(val_loss)
        
        print(f'Epoch: {epoch+1}/{epochs}')
        print(f'Training Loss: {running_loss/len(train_loader):.3f} | Training Acc: {train_acc:.2f}%')
        print(f'Validation Loss: {val_loss/len(val_loader):.3f} | Validation Acc: {val_acc:.2f}%')
        print('--------------------')
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            
    return model

In [28]:
# Preparación de datos
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(X_train, dtype=torch.float32).permute(0, 3, 1, 2),
    torch.tensor(y_train.argmax(axis=1), dtype=torch.long)
)

val_dataset = torch.utils.data.TensorDataset(
    torch.tensor(X_test, dtype=torch.float32).permute(0, 3, 1, 2),
    torch.tensor(y_test.argmax(axis=1), dtype=torch.long)
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64)

# Entrenar el modelo
model = ImprovedNeuralNetwork().to(device)
model = train_model(model, train_loader, val_loader, epochs=20, device=device)

Epoch: 1/20
Training Loss: 0.234 | Training Acc: 92.58%
Validation Loss: 0.052 | Validation Acc: 98.43%
--------------------
Epoch: 2/20
Training Loss: 0.089 | Training Acc: 97.18%
Validation Loss: 0.059 | Validation Acc: 98.33%
--------------------
Epoch: 3/20
Training Loss: 0.063 | Training Acc: 98.02%
Validation Loss: 0.044 | Validation Acc: 98.56%
--------------------
Epoch: 4/20
Training Loss: 0.054 | Training Acc: 98.30%
Validation Loss: 0.033 | Validation Acc: 98.90%
--------------------
Epoch: 5/20
Training Loss: 0.047 | Training Acc: 98.57%
Validation Loss: 0.046 | Validation Acc: 98.55%
--------------------
Epoch: 6/20
Training Loss: 0.041 | Training Acc: 98.72%
Validation Loss: 0.042 | Validation Acc: 98.62%
--------------------
Epoch: 7/20
Training Loss: 0.036 | Training Acc: 98.83%
Validation Loss: 0.036 | Validation Acc: 98.86%
--------------------
Epoch: 8/20
Training Loss: 0.035 | Training Acc: 98.90%
Validation Loss: 0.029 | Validation Acc: 99.17%
--------------------


In [30]:
# Prepare the test dataset
test_samples = test_df.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0
# Reorder dimensions to match PyTorch's expected format: [batch, channels, height, width]
test_samples = np.transpose(test_samples, (0, 3, 1, 2))

# Convert to tensor and move to device
test_tensor = torch.tensor(test_samples, dtype=torch.float32).to(device)

# Load the best model if you saved it during training
model.load_state_dict(torch.load('best_model.pth'))
model.to(device)

# Make predictions
model.eval()
predictions = []

# Process in batches to avoid memory issues
batch_size = 64
with torch.no_grad():
    for i in range(0, len(test_tensor), batch_size):
        batch = test_tensor[i:i+batch_size]
        outputs = model(batch)
        pred_probab = nn.Softmax(dim=1)(outputs)
        batch_preds = pred_probab.argmax(1).cpu().numpy()
        predictions.extend(batch_preds)

# Create submission file
submission_df = pd.DataFrame({
    'ImageId': range(1, len(predictions) + 1),
    'Label': predictions
})

# Save predictions
submission_df.to_csv('submission.csv', index=False)

# Print some statistics
print(f"Total predictions made: {len(predictions)}")
print("\nPrediction distribution:")
for label in range(10):
    count = (np.array(predictions) == label).sum()
    percentage = (count / len(predictions)) * 100
    print(f"Digit {label}: {count} images ({percentage:.1f}%)")

Total predictions made: 28000

Prediction distribution:
Digit 0: 2778 images (9.9%)
Digit 1: 3196 images (11.4%)
Digit 2: 2818 images (10.1%)
Digit 3: 2779 images (9.9%)
Digit 4: 2759 images (9.9%)
Digit 5: 2522 images (9.0%)
Digit 6: 2745 images (9.8%)
Digit 7: 2884 images (10.3%)
Digit 8: 2749 images (9.8%)
Digit 9: 2770 images (9.9%)
